In [1]:
# import random
# import sys
# import threading


import gym
from gym import spaces
from gym.utils import seeding

import numpy as np

import tensorflow as tf

In [2]:
import game2048
import rl

In [3]:
env = game2048.Game2048()

In [11]:
def Networks(env, states, hidden_num):
    """Maps states tensor to the pair of policy and value tensors."""
    with tf.device("/cpu:0"):
        with tf.variable_scope('common'):
            hidden = tf.reshape(states, [-1, 16])
            for num in hidden_num:
                hidden = tf.contrib.layers.relu(
                    hidden, num,
                    weights_initializer=tf.truncated_normal_initializer(stddev=1.),
                    biases_initializer=tf.constant_initializer(0.02))

        with tf.variable_scope('policy'):
            logits = tf.contrib.layers.linear(hidden, env.action_space.n)
        with tf.variable_scope('value'):
            value_out = tf.contrib.layers.linear(hidden, 1)
        return logits, value_out

In [17]:
options = dict(rl.DEFAULT_OPTIONS)
options['threads'] = 4
options['learning_rate'] = 0.0001

In [18]:
ac = rl.ActorCritic(game2048.Game2048,
                    lambda env, st: Networks(env, st, [20, 20, 20]),
                    options=options)

In [19]:
logs_dir = '/media/vertix/UHDD/tmp/tensorflow_logs/{}/{:02d}'.format('2048', 3)
ac.Init(logs_dir)

In [20]:
ac.Learn(10000000)

In [16]:
ac.Close()